In [1]:
!set -x\
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 16.3 MB/s eta 0:00:00
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.6 LTS"
/etc/os-release:NAME="Ubuntu"
/etc/os-relea

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from torch import nn
import torch
import numpy as np
import math

# scaled dot product


In [4]:
class scaled_dot_product(nn.Module):
  def __init__(self):
    super(scaled_dot_product, self).__init__()
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, query, key, value, mask, masked):
    key_tr = torch.transpose(key, 2, 3)
    dk = torch.tensor(query.size()[-1])
    outputs = (query @ key_tr) / dk.sqrt()

    if mask is not None:
      outputs = outputs.masked_fill(mask == 0, -2**30)

    if masked:
      mask1 = torch.ones_like(outputs[:,:,:])
      mask1 = torch.triu(mask1, diagonal=1)
      mask1 = mask1*(-2**30)
      padding = torch.ones_like(mask1)
      padding = torch.tril(padding)
      outputs = torch.where(padding==1, outputs, mask1)

    attention_map = self.softmax(outputs)
    scaled_dp = (attention_map @ value)
    return scaled_dp, attention_map

# multihead attention

In [5]:
class multihead_attention(nn.Module):
  def __init__(self, embedding_dims, model_dims, num_heads):
    super(multihead_attention, self).__init__()

    self.num_heads = num_heads
    self.linear_q = nn.Linear(embedding_dims, model_dims)
    self.linear_k = nn.Linear(embedding_dims, model_dims)
    self.linear_v = nn.Linear(embedding_dims, model_dims)
    self.linear_output = nn.Linear(model_dims, model_dims)
    self.attention = scaled_dot_product()

  def split(self, tensor):
    batch_size, sequence_length, d_model = tensor.size()

    d_tensor = d_model // self.num_heads
    tensor = tensor.view(batch_size, sequence_length, self.num_heads, d_tensor).transpose(1, 2)
    return tensor

  def cat(self, tensor):
    batch_size, self.num_heads, sequence_length, d_tensor = tensor.size()
    d_model = self.num_heads * d_tensor

    tensor = tensor.transpose(1, 2).contiguous().view(batch_size, sequence_length, d_model)
    return tensor

  def forward(self, query, key, value, mask, masked):
    query1 = self.linear_q(query)
    key1 = self.linear_k(key)
    value1 = self.linear_v(value)

    query_split = self.split(query1)
    key_split = self.split(key1)
    value_split = self.split(value1)

    multihead_attn, attn_map = self.attention(query_split, key_split, value_split, mask, masked)
    attn_outputs1 = self.cat(multihead_attn)
    attn_outputs2 = self.linear_output(attn_outputs1)
    return attn_outputs2

# feed forward 

In [6]:
class feed_forward(nn.Module):
  def __init__(self, model_dims, ff_dims):
    super(feed_forward, self).__init__()
    self.dropout_p = 0.1
    self.linear_inner = nn.Linear(model_dims, ff_dims)
    self.linear_outer = nn.Linear(ff_dims, model_dims)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=self.dropout_p)

  def forward(self, input):
    output1 = self.linear_inner(input)
    output2 = self.relu(output1)
    output3 = self.dropout(output2)
    output4 = self.linear_outer(output3)
    return output4


# layer norm

In [7]:
class layer_norm(nn.Module):
  def __init__(self, model_dims, eps=1e-6):
    super(layer_norm, self).__init__()
    self.layernorm = nn.LayerNorm(model_dims)

  def forward(self, input):
    output = self.layernorm(input)
    return output

# sublayer connection

In [8]:
class sublayer_connection(nn.Module):
  def __init__(self, model_dims, dropout_p):
    super(sublayer_connection, self).__init__()
    self.layernorm = layer_norm(model_dims)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self, input, output):
    outputs = self.layernorm(input + self.dropout(output))
    return outputs

# encoder module

In [9]:
class encoder_module(nn.Module):
  def __init__(self, model_dims, ff_dims, dropout_p, embedding_dims, num_heads):
    super(encoder_module, self).__init__()
    self.multiheadattention_en = multihead_attention(embedding_dims, model_dims, num_heads)
    self.feedforward = feed_forward(model_dims, ff_dims)
    self.sublayerconnection1 = sublayer_connection(model_dims, dropout_p)
    self.sublayerconnection2 = sublayer_connection(model_dims, dropout_p)

  def forward(self, encoder_inputs, mask):
    multihead_a = self.sublayerconnection1(encoder_inputs, self.multiheadattention_en(encoder_inputs, encoder_inputs, encoder_inputs, mask, masked=False))
    outputs = self.sublayerconnection2(multihead_a, self.feedforward(multihead_a))
    return outputs

# decoder module

In [10]:
class decoder_module(nn.Module):
  def __init__(self, model_dims, ff_dims, dropout_p, embedding_dims, num_heads):
    super(decoder_module, self).__init__()
    self.multiheadattention_en = multihead_attention(embedding_dims, model_dims, num_heads)
    self.multiheadattention_ende = multihead_attention(embedding_dims, model_dims, num_heads)
    self.feedforward = feed_forward(model_dims, ff_dims)
    self.sublayerconnection1 = sublayer_connection(model_dims, dropout_p)
    self.sublayerconnection2 = sublayer_connection(model_dims, dropout_p)
    self.sublayerconnection3 = sublayer_connection(model_dims, dropout_p)

  def forward(self, decoder_inputs, encoder_outputs, mask):
    masked_multihead_attn = self.sublayerconnection1(decoder_inputs, self.multiheadattention_en(decoder_inputs, decoder_inputs, decoder_inputs, mask, masked=True)) # self attention 
    outputs1 = self.sublayerconnection2(masked_multihead_attn, self.multiheadattention_ende(masked_multihead_attn, encoder_outputs, encoder_outputs, mask, masked=False)) # encoder-decoder attention
    outputs2 = self.sublayerconnection3(outputs1, self.feedforward(outputs1))
    return outputs2

# encoder

In [11]:
class encoder(nn.Module):
  def __init__(self, model_dims, ff_dims, num_heads, num_layers, embedding_dims, len_vocab, dropout_p):
    super(encoder, self).__init__()
    self.positionalencoding = positional_encoding()
    self.embedding = nn.Embedding(len_vocab, embedding_dims)
    self.encoder_layer = nn.ModuleList([encoder_module(model_dims = model_dims,
                                                       ff_dims = ff_dims,
                                                       num_heads = num_heads,
                                                       dropout_p=dropout_p,
                                                       embedding_dims = embedding_dims)
                                        for _ in range(num_layers)])
    

  def forward(self, encoder_inputs, mask):
    encoder_embedding = self.embedding(encoder_inputs)
    encoder_inputs_position = encoder_embedding + self.positionalencoding(encoder_embedding)
    outputs = encoder_inputs_position
    for layer in self.encoder_layer:
      outputs = layer(outputs, mask)
    return outputs
    

# decoder

In [12]:
class decoder(nn.Module):
  def __init__(self, model_dims, ff_dims, num_heads, num_layers, embedding_dims, len_vocab, dropout_p):
    super(decoder, self).__init__()

    self.positionalencoding = positional_encoding()
    self.embedding = nn.Embedding(len_vocab, embedding_dims)
    self.last_linear = nn.Linear(model_dims, len_vocab)
    self.decoder_layer = nn.ModuleList([decoder_module(model_dims = model_dims,
                                                       ff_dims = ff_dims,
                                                       num_heads = num_heads,
                                                       dropout_p=dropout_p,
                                                       embedding_dims = embedding_dims)
                                        for _ in range(num_layers)])
    
  def forward(self, decoder_inputs, encoder_outputs, mask):
    decoder_embedding = self.embedding(decoder_inputs)
    decoder_inputs_position = decoder_embedding + self.positionalencoding(decoder_embedding)
    outputs = decoder_inputs_position
    for layer in self.decoder_layer:
      outputs = layer(outputs, encoder_outputs, mask)
    outputs2 = self.last_linear(outputs)
    return outputs2

# positional encoding

In [13]:
class positional_encoding(nn.Module):
  def __init__(self):
    super(positional_encoding, self).__init__()


  def forward(self, inputs):
    batch, max_length, model_dims = inputs.size() 
    position_vec = np.array([pos/np.power(10000, 2*i/model_dims) for pos in range(max_length) for i in range(model_dims)])
    position_vec[::2] = np.sin(position_vec[::2])
    position_vec[1::2] = np.cos(position_vec[1::2])
    positional_matrix = torch.from_numpy(position_vec).reshape(max_length, model_dims).type(torch.float32)
    positional_matrix.requires_grad = False
    return positional_matrix.to(device)

# transformer

In [14]:
class transformer(nn.Module):
  def __init__(self, model_dims, ff_dims, num_heads, num_layers, embedding_dims, len_vocab, dropout_p, src_pad_idx):
    super(transformer, self).__init__()
    self.src_pad_idx = src_pad_idx
    self.encoder = encoder(model_dims, ff_dims, num_heads, num_layers, embedding_dims, len_vocab, dropout_p)
    self.decoder = decoder(model_dims, ff_dims, num_heads, num_layers, embedding_dims, len_vocab, dropout_p)

  def forward(self, encoder_inputs, decoder_inputs):
    encoder_mask = self.make_pad_mask(encoder_inputs, encoder_inputs)
    decoder_mask = self.make_pad_mask(decoder_inputs, decoder_inputs)
    encoder_outputs = self.encoder(encoder_inputs, encoder_mask)
    decoder_outputs =self.decoder(decoder_inputs, encoder_outputs, decoder_mask)
    return decoder_outputs

  def make_pad_mask(self,q,k):
    len_q,len_k = q.size(1),k.size(1)
    k = k.ne(self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    k = k.repeat(1,1,len_q,1)
    q = q.ne(self.src_pad_idx).unsqueeze(1).unsqueeze(3)
    q = q.repeat(1,1,1,len_k)
    mask = k & q
    return mask


# chatbot data

In [15]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import re
import urllib.request
import time
 
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
data = pd.read_csv('ChatBotData.csv')
data.head()
#data = data.sample(frac=1)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [16]:
print('챗봇 샘플의 개수 :', len(data))

챗봇 샘플의 개수 : 11823


In [17]:
print(data.isnull().sum())

Q        0
A        0
label    0
dtype: int64


In [18]:
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


# data preprocess

In [19]:
from konlpy.tag import Mecab

filters = "([~.!?\"':;)(])"
CHANGE_FILTER = re.compile(filters)
tokenizer = Mecab()

pad = '[PAD]'
unk = '[UNK]'
start = '[BOS]'
end = '[END]'
token = [pad, start, end, unk]

In [20]:
def prepro_morphs(data):
  sentences = []
  for sentence in data:
    sentence = re.sub(CHANGE_FILTER, "", sentence)
    sentence = sentence.strip()
    sentence = ' '.join(tokenizer.morphs(sentence))
    sentences.append(sentence)
  return sentences

In [21]:
prepro_morphs(data['Q'])[:10]

['12 시 땡',
 '1 지망 학교 떨어졌 어',
 '3 박 4 일 놀 러 가 고 싶 다',
 '3 박 4 일 정도 놀 러 가 고 싶 다',
 'PPL 심하 네',
 'SD 카드 망가졌 어',
 'SD 카드 안 돼',
 'SNS 맞 팔 왜 안 하 지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아 는데 매일 하 는 중',
 'SNS 시간 낭비 인데 자꾸 보 게 됨']

In [22]:
def load_vocab(data, token):
  question = prepro_morphs(data['Q'])
  answer = prepro_morphs(data['A'])
  sentences = []
  sentences.extend(question)
  sentences.extend(answer)
  tokenizer = Mecab()
  vocab = []
  for s in sentences:
    words = s.split(' ')
    vocab.extend(words)
  vocab = list(set(vocab))
  vocab = token + vocab 
  i2v = {}
  v2i = {}
  for i, v in enumerate(vocab):
    i2v[i] = v 
  for i, v in i2v.items():
    v2i[v] = i
  return i2v, v2i, len(i2v)

In [23]:
index2word, word2index, vocab_size = load_vocab(data, token)

In [24]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/데이터사이언스 특론/실습/index2word.pickle', 'wb') as handle:
    pickle.dump(index2word, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/Colab Notebooks/데이터사이언스 특론/실습/word2index.pickle', 'wb') as handle:
    pickle.dump(word2index, handle, protocol=pickle.HIGHEST_PROTOCOL)
  
with open('/content/drive/MyDrive/Colab Notebooks/데이터사이언스 특론/실습/vocab_size.pickle', 'wb') as handle:
    pickle.dump(vocab_size, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
def encoder_input_preprocessing(data, vocab, max_length): # max_length 만큼 padding만 처리
  questions = prepro_morphs(data['Q'])
  answers = prepro_morphs(data['A'])

  questions_index = []
  answers_index = []

  for q in questions:
    q = [start] + q.split(' ') + [end]
    if len(q) >= max_length:
      q = q[:max_length-1] + [end]
    else:
      q = q + [pad]*(max_length - len(q))
    q_index = [vocab[w] for w in q]
    questions_index.append(torch.tensor(q_index))

  for a in answers:
    a = [start] + a.split(' ') + [end] 
    if len(a) >= max_length:
      a = a[:max_length-1] + [end] 
    else:
      a = a + [pad]*(max_length - len(a))
    a_index = [vocab[w] for w in a]
    answers_index.append(torch.tensor(a_index))
  return questions_index, answers_index

In [26]:
def decoder_input_preprocessing(data, vocab, max_length): # max_length 만큼 padding + start token
  questions = prepro_morphs(data['Q'])
  answers = prepro_morphs(data['A'])

  questions_index = []
  answers_index = []

  for q in questions:
    q = [start] + q.split(' ')
    if len(q) >= max_length:
      q = q[:max_length]
    else:
      q = q + [pad]*(max_length - len(q))
    q_index = [vocab[w] for w in q]
    questions_index.append(torch.tensor(q_index))

  for a in answers:
    a = [start] + a.split(' ')
    if len(a) >= max_length:
      a = a[:max_length]
    else:
      a = a + [pad]*(max_length - len(a))
    a_index = [vocab[w] for w in a]
    answers_index.append(torch.tensor(a_index))
  return questions_index, answers_index

In [27]:
def decoder_target_preprocessing(data, vocab, max_length): # padding + end token
  questions = prepro_morphs(data['Q'])
  answers = prepro_morphs(data['A'])

  questions_index = []
  answers_index = []

  for q in questions:
    q = q.split(' ') + [end]
    if len(q) >= max_length:
      q = q[:max_length-1] + [end]
    else:
      q = q + [pad]*(max_length - len(q))
    q_index = [vocab[w] for w in q]
    questions_index.append(torch.tensor(q_index))

  for a in answers:
    a = a.split(' ') + [end]
    if len(a) >= max_length:
      a = a[:max_length-1] + [end]
    else:
      a = a + [pad]*(max_length - len(a))
    a_index = [vocab[w] for w in a]
    answers_index.append(torch.tensor(a_index))
  return questions_index, answers_index

In [28]:
from torch.utils.data import Dataset

class chatbot_dataset(Dataset):
    def __init__(self, data):
      self.encoder_input_q, self.encoder_input_a = encoder_input_preprocessing(data, word2index, 15)
      self.decoder_input_q, self.decoder_input_a = decoder_input_preprocessing(data, word2index, 15)
      self.decoder_target_q, self.decoder_target_a = decoder_target_preprocessing(data, word2index, 15)

    def __len__(self):
      return len(self.encoder_input_q)
    
    def __getitem__(self, index):
        return [
            self.encoder_input_q[index],
            self.decoder_input_a[index],
            self.decoder_target_a[index]
        ]

In [29]:
chatbot_data = chatbot_dataset(data)

# dataloader

In [30]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def collate_fn(batch):
  encoder_input_data = pad_sequence([b[0] for b in batch], batch_first=True)
  decoder_input_data = pad_sequence([b[1] for b in batch], batch_first=True)
  decoder_target_data = pad_sequence([b[2] for b in batch], batch_first=True)
  return encoder_input_data, decoder_input_data, decoder_target_data

input_dataloader = DataLoader(chatbot_data, collate_fn=collate_fn, batch_size=32)

# train

In [31]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [32]:
transformer_model = transformer(512, 1024, 8, 6, 512, vocab_size, 0.1, 0).to(device)

In [33]:
def train(model, optimizer, criterion, dataloader):
  losses = [] 
  for data in dataloader:
    optimizer.zero_grad()

    encoder_input, decoder_input, decoder_target = data
    encoder_input = encoder_input.to(device)
    decoder_input = decoder_input.to(device)
    decoder_target = decoder_target.to(device)

    output = model(encoder_input, decoder_input).to(device)
    loss = criterion(output.transpose(1,2), decoder_target)
    losses.append(loss.item())
    
    loss.backward()
    optimizer.step()

  return sum(losses) / len(losses)



In [34]:
epochs = 150
lr = 0.0001
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [35]:
for i in range(epochs):
  avg_loss = train(transformer_model, optimizer, criterion, input_dataloader)
  print('epoch = ', i, ' avg_loss = ', avg_loss)

epoch =  0  avg_loss =  2.700902548673991
epoch =  1  avg_loss =  2.058935417033531
epoch =  2  avg_loss =  1.8141097181552166
epoch =  3  avg_loss =  1.615794359187822
epoch =  4  avg_loss =  1.4346207233699593
epoch =  5  avg_loss =  1.2648930072784423
epoch =  6  avg_loss =  1.1000548454555306
epoch =  7  avg_loss =  0.9266320613590446
epoch =  8  avg_loss =  0.7545096521442001
epoch =  9  avg_loss =  0.5996167026661537
epoch =  10  avg_loss =  0.4630675696037911
epoch =  11  avg_loss =  0.3454810722454174
epoch =  12  avg_loss =  0.2506240848149802
epoch =  13  avg_loss =  0.17038589554864006
epoch =  14  avg_loss =  0.11888661252687106
epoch =  15  avg_loss =  0.08321706364686425
epoch =  16  avg_loss =  0.06366445044609341
epoch =  17  avg_loss =  0.050813221037891265
epoch =  18  avg_loss =  0.04790099659884298
epoch =  19  avg_loss =  0.039811701012926325
epoch =  20  avg_loss =  0.03758916434001278
epoch =  21  avg_loss =  0.033511475315065806
epoch =  22  avg_loss =  0.033005

In [36]:
torch.save(transformer_model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/데이터사이언스 특론/실습/transformer.bin')

# predict

In [37]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [38]:
from konlpy.tag import Mecab
import re
import torch

filters = "([~.!?\"':;)(])"
CHANGE_FILTER = re.compile(filters)
tokenizer = Mecab()

pad = '[PAD]'
unk = '[UNK]'
start = '[BOS]'
end = '[END]'

In [39]:
def prepro_morphs_sequence(sentence, tokenizer):
  sentence = re.sub(CHANGE_FILTER, "", sentence)
  sentence = sentence.strip()
  sentence = ' '.join(tokenizer.morphs(sentence))
  return sentence

In [41]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/데이터사이언스 특론/실습/index2word.pickle', 'rb') as handle:
  index2word = pickle.load(handle)
with open('/content/drive/MyDrive/Colab Notebooks/데이터사이언스 특론/실습/word2index.pickle', 'rb') as handle:
  word2index = pickle.load(handle)
with open('/content/drive/MyDrive/Colab Notebooks/데이터사이언스 특론/실습/vocab_size.pickle', 'rb') as handle:
  vocab_size = pickle.load(handle)

In [42]:
model = transformer_model = transformer(512, 1024, 8, 6, 512, vocab_size, 0.1, 0).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/데이터사이언스 특론/실습/transformer.bin'))

<All keys matched successfully>

In [43]:
def make_pad_mask(q,k):
  
  len_q,len_k = q.size(1),k.size(1)
  k = k.ne(0).unsqueeze(1).unsqueeze(2)
  k = k.repeat(1,1,len_q,1)
  q = q.ne(0).unsqueeze(1).unsqueeze(3)
  q = q.repeat(1,1,1,len_k)
  mask = k & q
  return mask

In [52]:
def chatbot(sequence):
  max_length = 25

  sequence = prepro_morphs_sequence(sequence, tokenizer)
  sequence_list = [[1] + [word2index[x] for x in sequence.split(' ')] + [2] + [0]*(max_length-len([1] + [word2index[x] for x in sequence.split(' ')] + [2]))]

  encoder_input = torch.tensor(sequence_list).to(device)
  encoder_mask = make_pad_mask(encoder_input, encoder_input)
  encoder_output = model.encoder(encoder_input, encoder_mask)

  decoder_input_list = [1]

  answer = ''
  while True:
    decoder_input = torch.tensor([decoder_input_list]).to(device)
    decoder_mask = make_pad_mask(decoder_input, decoder_input)
    decoder_output = model.decoder(decoder_input, encoder_output, None)
    decoder_output = nn.Softmax(dim=-1)(decoder_output)
    num = decoder_output.argmax(-1)[0].tolist()[-1]
    decoder_input_list.append(num)
    if (num == 2) or (len(decoder_input_list) == 25):
      break
    if num != 0:
      answer = answer + ' ' +index2word[num]
      
  return answer


In [60]:
chatbot('너는 누구니?')

' 저 는 위로 해 드리 는 로봇 이 에요 어제 와 내일 너무 자책'

In [59]:
chatbot('내일 날씨 어때?')

' 날씨 어 플 에 물 어 보 는 것 도 좋 은 방법 이'

In [62]:
chatbot('놀고 싶다')

' 저 는 마음 을 이어주 는 위 로 봇 과 비교 하 지'